In [3]:
import numpy as np

In [4]:
# helix p_alpha relative probabilities
p_a = {
    'E': 1.53, 'A': 1.45, 'L': 1.34, 'H': 1.24, 'M': 1.20, 'Q': 1.17, 'W': 1.14, 'V': 1.14, 'F': 1.12,  # builders
    'K': 1.07, 'I': 1.00, 'D': 0.98, 'T': 0.82, 'S': 0.79, 'R': 0.79, 'C': 0.77,  # indifferent
    'N': 0.73, 'Y': 0.61, 'P': 0.59, 'G': 0.53  # breakers
}

# helix builder/indifferent/breaker class weights
w_a = {aa: (1 if score > 1.1 else -1 if score < 0.75 else 0.5) for aa, score in p_a.items()}

# strand p_beta relative probabilities
p_b = {
    'M': 1.67, 'V': 1.65, 'I': 1.60, 'C': 1.30, 'Y': 1.29, 'F': 1.28, 'Q': 1.23,  'L': 1.22, 'T': 1.20, 'W': 1.19,  # builders
    'A': 0.97, 'R': 0.90, 'G': 0.81, 'D': 0.80,  # indifferent
    'K': 0.74, 'S': 0.72, 'H': 0.71, 'N': 0.65, 'P': 0.62, 'E': 0.26  # breakers
}

# strand builder/indifferent/breaker class weights
# changed to 2/0/-1 encoding for convenience, so that a window of 5 is a strand core if sum(window) >= 5.
w_b = {aa: (2 if score > 1 else -1 if score < 0.78 else 0) for aa, score in p_b.items()}

# sequence of 5JXV
seq    = 'MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE'
ss_ref = '-SSSSSSS----SSSSSSS---HHHHHHHHHHHHHH-----SSSSS----SSSSS-'
he_ref = '--------------HHHHHHHHHHHHHHHHHH------------HHHHHHHHHHHH'
sh_ref = 'SSSSSSSSS----SSSSSSSS------SSSSSSSS-----SSSSSSSSSSSSSSSS'
pred   = 'SSSSSSSSS----SHHHHHHHHHHHHHSSSSSSSS-----SSSSSSSSSSSSSSSS'

# reference secondary structure for accuracy calculation
ss_ref = '-SSSSSSS----SSSSSSS---HHHHHHHHHHHHHH-----SSSSS----SSSSS-'
he_ref = '--------------HHHHHHHHHHHHHHHHHH------------HHHHHHHHHHH-'
sh_ref = 'SSSSSSSSS----SSSSSSSS------SSSSSSSS-----SSSSSSSSSSSSSSSS'

In [18]:
def find_helix(seq):
    seq_length = len(seq)
    # Generate an array containing score for every aa in the sequence
    weights = []
    probability = []
    helix = ["-" for _ in range(seq_length)]
    for i in range (0, seq_length):
        weights.append(w_a[seq[i]])
        probability.append(p_a[seq[i]])
    for i in range(0, seq_length-6):
        weight = np.sum(weights[i:i+6])
        if weight >= 4:
            # extend left bond
            if i > 0:
                left_bound = i-1
                while (np.sum(probability[left_bound:left_bound+4])>4):
                    if left_bound >= 0:
                        left_bound -= 1
                    else: 
                        break
            else:
                left_bound = i
            if i+7 < seq_length:
                right_bound = i+7
                while(np.sum(probability[right_bound-4:right_bound])>4):
                    if right_bound <= seq_length:
                        right_bound += 1
                    else:
                        break
            else: 
                right_bound = seq_length
            for j in range(left_bound+1, right_bound-1):
                helix[j] = "H"
    return helix

def find_strands(seq):
    seq_length = len(seq)
    # Generate an array containing score for every aa in the sequence
    weights = []
    probability = []
    sheet = ['-' for _ in range(seq_length)]
    for i in range (0, seq_length):
        weights.append(w_b[seq[i]])
        probability.append(p_b[seq[i]])
    for i in range(0, seq_length-5):
        weight = np.sum(weights[i:i+5])
        if weight >= 5:
            # extend left bond
            if i-1>=0:
                left_bound = i-1
                while (np.sum(probability[left_bound:left_bound+4]) > 4):
                    if left_bound >= 0:
                        left_bound -= 1
                    else: 
                        break
            else:
                left_bound = i
            if i + 6 <=seq_length:
                right_bound = i+6
                while(np.sum(probability[right_bound-4:right_bound])>4):
                    if right_bound <= seq_length:
                        right_bound += 1
                    else:
                        break
            else: 
                right_bound = seq_length
            for j in range(left_bound+1, right_bound-1):
                sheet[j] = 'S'
    return sheet

In [19]:
print(''.join(find_helix(seq)))

--------------HHHHHHHHHHHHHHHHHH------------HHHHHHHHHHH-


In [20]:
print(''.join(find_strands(seq)))

SSSSSSSSS----SSSSSSSS------SSSSSSSS-----SSSSSSSSSSSSSSSS


In [37]:
temp_seq = 'ETTTEAV'
w = 0
p = 0
for i in temp_seq:
    w += w_b[i]
    p += p_b[i]
print(w)
print(p)

6
6.74


In [38]:
temp_seq = 'ETTTEAV'
w = 0
p = 0
for i in temp_seq:
    w += w_a[i]
    p += p_a[i]
print(w)
print(p)

5.5
8.11


In [39]:
def find_helix_(seq):
    seq_length = len(seq)
    # Generate an array containing score for every aa in the sequence
    weights = []
    probability = []
    helix = ["-" for _ in range(seq_length)]
    for i in range (0, seq_length):
        weights.append(w_a[seq[i]])
        probability.append(p_a[seq[i]])
    for i in range(0, seq_length-6):
        weight = np.sum(weights[i:i+6])
        if weight >= 4:
            # extend left bond
            left_bound = i
            while True:
                if left_bound-1<0 or left_bound+3>seq_length:
                    break
                elif  (np.sum(probability[left_bound-1:left_bound+3]) >= 4):
                    left_bound -= 1
                else:
                    break
            right_bound = i+6
            while True:
                if right_bound-3<0 or right_bound+1>seq_length:
                    break
                elif (np.sum(probability[right_bound-3:right_bound+1])>=4):
                    right_bound += 1
                else:
                    break
            for j in range(left_bound, right_bound):
                helix[j] = "H"
    return helix

def find_strands_(seq):
    seq_length = len(seq)
    # Generate an array containing score for every aa in the sequence
    weights = []
    probability = []
    sheet = ['-' for _ in range(seq_length)]
    for i in range (0, seq_length):
        weights.append(w_b[seq[i]])
        probability.append(p_b[seq[i]])
    for i in range(0, seq_length-5):
        weight = np.sum(weights[i:i+5])
        if weight >= 5:
            # extend left bond
            left_bound = i
            while True:
                if left_bound-1<0 or left_bound+3>seq_length:
                    break
                elif  (np.sum(probability[left_bound-1:left_bound+3]) >= 4):
                    left_bound -= 1
                else:
                    break
            right_bound = i+5
            while True:
                if right_bound-3<0 or right_bound+1>seq_length:
                    break
                elif (np.sum(probability[right_bound-3:right_bound+1])>=4):
                    right_bound += 1
                else:
                    break
            for j in range(left_bound, right_bound):
                sheet[j] = 'S'
    return sheet

In [40]:
print(''.join(find_helix_(seq)))

--------------HHHHHHHHHHHHHHHHHH------------HHHHHHHHHHHH


In [41]:
print(''.join(find_strands_(seq)))

SSSSSSSSS----SSSSSSSS------SSSSSSSS-----SSSSSSSSSSSSSSSS


In [43]:
he = find_helix_(seq)
st = find_strands_(seq)
pred = []
i = 0
while i < len(seq):
    if(he[i]=='H' and st[i]=='S'):
        end = i+1
        if end < len(seq):
            while(he[end]=='H' and st[end]=='S'):
                end += 1
                if end >= len(seq):
                    break
        p_h = 0
        p_s = 0
        for j in range(i,end):
            p_h += p_a[seq[j]]
            p_s += p_b[seq[j]]
        if p_h > p_s:
            for _ in range(i, end):
                pred.append('H') 
        else:
            for _ in range(i, end):
                pred.append('S')
        i = end
    elif(he[i]=='H' and st[i]!='S'):
        pred.append('H')
        i += 1
    elif(st[i]=='S'and he[i]!='H'):
        pred.append('S')
        i += 1
    else:
        pred.append('-')
        i += 1
pred = ''.join(pred)

SSSSSSSSS----SHHHHHHHHHHHHHSSSSSSSS-----SSSSSSSSSSSSSSSS


In [44]:
counter = 0
for i in range(len(pred)):
    if (pred[i]==ss_ref[i] and pred[i]!='-'):
        counter += 1
sim = counter / len(pred)
print(sim) 

0.4107142857142857


In [45]:
counter

23

In [46]:
len(pred)

56